<a href="https://colab.research.google.com/github/arunbretlee/chatbot/blob/main/Fake_News_Dection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***FAKE NEWS DECTECTION***

#Importing required libraries

In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string
import matplotlib.pyplot as plt

#Importing the datasets


In [20]:
try:
    df_fake = pd.read_csv("Fake.csv", engine='python', on_bad_lines='skip')
except Exception as e:
    print(f"Error reading Fake.csv: {e}")
    # If the above still fails, inspect the problematic row manually.
    # You might need to open the CSV file in a text editor and go to row 2388
    # to see the malformed string.

# Read the other file as before
try:
    df_true = pd.read_csv("True.csv", engine='python', on_bad_lines='skip')
except Exception as e:
    print(f"Error reading True.csv: {e}")

In [21]:
df_fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [22]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [23]:
df_fake["class"] = 0
df_true["class"] = 1

In [24]:
df_fake.shape, df_true.shape

((14568, 5), (14107, 5))

In [26]:
df_fake_manualTesting = df_fake.tail(10)
for i in range(14568, 14568, -1):
  df_fake.drop([i], axis = 0, inplace=True)


df_true_manualTesting = df_true.tail(10)
for i in range(14107, 14107, -1):
  df_true.drop([i], axis = 0, inplace=True)

In [27]:
df_fake.shape, df_true.shape

((14568, 5), (14107, 5))

In [28]:
df_fake_manualTesting["class"] = 0
df_true_manualTesting["class"] = 1

<ipython-input-28-cde1a4f69a40>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manualTesting["class"] = 0
<ipython-input-28-cde1a4f69a40>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manualTesting["class"] = 1


In [29]:
df_fake_manualTesting.head(10)

,title,text,subject,date,class
14558,KEY TERROR GROUPS EMERGE As U.S. Grip On Afgha...,Obama pulled troops out so this created a vacu...,politics,"Feb 1, 2016",0
14559,GOOD GRIEF! HILLARY TELLS ANOTHER WHOPPER Just...,Ed Henry of FOX News tells us tonight that Hil...,politics,"Jan 31, 2016",0
14560,“TEFLON” TRUMP WINS SUPPORT From Unlikely Grou...,Trump is winning support from latinos and from...,politics,"Jan 31, 2016",0
14561,BREAKING NEWS: Obama To Meet With Special Gues...,Has our lawless President finally conceded tha...,politics,"Jan 31, 2016",0
14562,HILLARY CLINTON FINALLY ASKED About The Non-Di...,Artful dodger Hillary Clinton dodges getting t...,politics,"Jan 31, 2016",0
14563,OBAMA TO VISIT MOSQUE WHERE RADICAL IMAM Condo...,Our Imam In Chief told America he was a Christ...,politics,"Jan 31, 2016",0
14564,OBAMA’S FIRST VISIT TO A U.S. MOSQUE Raises Qu...,Adding insult to injury is what Obama does bes...,politics,"Jan 30, 2016",0
14565,UPDATE: CRUZ COMMENTS On His Desperate Vote Sh...,UPDATE:Texas Sen. Ted Cruz is defending a camp...,politics,"Jan 30, 2016",0
14566,HYSTERICAL…THE DEMOCRAT CONVENTION Schedule Is...,LOL! You ll want to share this with everyone D...,politics,"Jan 30, 2016",0
14567,OBAMA THROWS HILLARY UNDER THE BUS: E-mail Sca...,Obama decides to throw Hillary under the bus I...,politics,"Jan 30, 2016",0


In [30]:
data_merge = pd.concat([df_fake, df_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [32]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [33]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [34]:
data.isnull().sum()

,0
text,0
class,0


In [35]:
data = data.sample(frac = 1)

In [36]:
data.head()

,text,class
12902,Who won the debate tonight? Donald J. Trump or...,0
9344,The 2017 Christmas decorations were unveiled i...,0
8866,WASHINGTON (Reuters) - In declining to seek pr...,1
13986,This is a shocker! The very dry and serious Ch...,0
10528,WASHINGTON (Reuters) - Republican 2012 preside...,1


In [37]:
data.reset_index(inplace = True)
data.drop(["index"], axis = 1, inplace = True)

In [38]:
data.columns

Index(['text', 'class'], dtype='object')

In [39]:
data.head()

,text,class
0,Who won the debate tonight? Donald J. Trump or...,0
1,The 2017 Christmas decorations were unveiled i...,0
2,WASHINGTON (Reuters) - In declining to seek pr...,1
3,This is a shocker! The very dry and serious Ch...,0
4,WASHINGTON (Reuters) - Republican 2012 preside...,1


In [76]:
def word(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('\\W', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [41]:
data['text'] = data['text'].apply(word)

In [43]:
X = data['text']
y = data['class']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
X_train = vectorization.fit_transform(X_train)
X_test = vectorization.transform(X_test)

#Logistic Regression


In [46]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression()

In [47]:
pred_LR = LR.predict(X_test)

In [48]:
LR.score(X_test, y_test)

0.9847956479285814

In [49]:
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98      3630
           1       0.98      0.99      0.98      3539

    accuracy                           0.98      7169
   macro avg       0.98      0.98      0.98      7169
weighted avg       0.98      0.98      0.98      7169



#Decision Tree Classifier

In [54]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [55]:
pred_DT = DT.predict(X_test)

In [56]:
DT.score(X_test, y_test)

0.9965127632863718

In [58]:
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3630
           1       1.00      1.00      1.00      3539

    accuracy                           1.00      7169
   macro avg       1.00      1.00      1.00      7169
weighted avg       1.00      1.00      1.00      7169



Greident Boosting Classifier

In [65]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state=0)
GB.fit(X_train, y_train)

GradientBoostingClassifier(random_state=0)

In [ ]:
pred_GB = GB.predict(X_test)

In [ ]:
GB.score(X_test, y_test)

In [ ]:
print(classification_report(y_test, pred_GB))


#Random Forest

In [61]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(random_state=0)
RF.fit(X_train, y_train)

RandomForestClassifier(random_state=0)

In [62]:
pred_RF = RF.predict(X_test)

In [63]:
RF.score(X_test, y_test)

0.979913516529502

In [64]:
print(classification_report(y_test, pred_RF))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3630
           1       0.98      0.98      0.98      3539

    accuracy                           0.98      7169
   macro avg       0.98      0.98      0.98      7169
weighted avg       0.98      0.98      0.98      7169



#Model Testing

In [66]:
def output_lable(n):
  if n == 0:
    return "Fake News"
  elif n == 1:
    return "True News"

In [73]:
def manual_testing(news):
  testing_news = {"text":[news]}
  new_def_test = pd.DataFrame(testing_news)
  new_def_test["text"] = new_def_test["text"].apply(word)

  new_X_test = new_def_test['text']
  new_XT_test = vectorization.transform(new_X_test)
  pred_LR = LR.predict(new_XT_test)
  pred_DT = DT.predict(new_XT_test)
  pred_GB = GB.predict(new_XT_test)
  pred_RF = RF.predict(new_XT_test)

  return print("\n\n LR Pred: {} \n DT Pred: {} \n GB Pred: {} \n RF Pred: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]),
                                                                                                              output_lable(pred_GB[0]),
                                                                                                              output_lable(pred_RF[0])))

In [75]:
# Get user input as a string. The input() function already returns a string.
latest_news = input("Enter the news article you want to test: ")

# Call the manual_testing function with the user's input
manual_testing(latest_news)

Enter the news article you want to test: The US state department has announced that it will impose sanctions on Sudan for the use of chemical weapons in 2024. Sudan became a party to the Chemical Weapons Convention in May 1999. The US said the country was in “non-compliance” with the provisions of the treaty and “will impose sanctions on Sudan, including restrictions on US exports to Sudan and on access to US government lines of credit.”  A statement added that the sanctions would take effect on 6 June and called on Sudan “to cease all chemical weapons use and uphold its obligations under the CWC.”


 LR Pred: True News 
 DT Pred: Fake News 
 GB Pred: Fake News 
 RF Pred: True News
